In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib

import harmonica as hm
import verde as vd
from skimage import data, img_as_float

import skimage.exposure

from skimage.filters import rank

import os
import zipfile
import shutil

import micromag as mg
import magali as ma
import time

## Generate dipole moments distribution

In [2]:
output_folder = "netcdf_files"
os.makedirs(output_folder, exist_ok=True)

In [3]:
vector_size = 2500
max_itr = 10

true_vector = np.empty((vector_size*max_itr,3))

for i in range(max_itr):  # Loop 10 times
    # Definir seed da aleatoriedade
    np.random.seed(i)
    dispersion = np.deg2rad(75)  # np.deg2rad(np.random.normal(0,10, vector_size)) ##
    
    alpha = np.deg2rad(np.random.uniform(0,360,vector_size))
    r = np.random.normal(0, dispersion, vector_size)
    
    x = np.sin(r)*np.cos(alpha)
    y = np.sin(r)*np.sin(alpha)
    z = np.cos(r)
    
    
    
    r_vector = np.array([x,y,z])
    
    amp = ma.amplitude_lognormal_distribution(vector_size)
    
    inc = 30
    dec = 30
    rotatated_vector = ma.rotate_vector(r_vector*amp, inc, dec)
    dipole_moments = np.copy(rotatated_vector.T)

    true_vector[i*vector_size:(i+1)*vector_size ,0] = dipole_moments[:,0]
    true_vector[i*vector_size:(i+1)*vector_size ,1] = dipole_moments[:,1]
    true_vector[i*vector_size:(i+1)*vector_size ,2] = dipole_moments[:,2]
    
    dipole_coordinates = (
        np.random.randint(0, 2000, vector_size),  # µm
        np.random.randint(0, 1400, vector_size),  # µm
        np.random.randint(-20, -1, vector_size),  # µm
    )
    
    
    sensor_sample_distance = 5.0  # µm
    coordinates = vd.grid_coordinates(
        region=[0, 2000, 0, 1400],  # µm
        spacing=2,  # µm
        extra_coords=sensor_sample_distance,
    )
    
    
    bz = mg.dipole_bz(coordinates, dipole_coordinates, dipole_moments)
    data = vd.make_xarray_grid(coordinates, bz, data_names=["bz"], dims=("y", "x"), extra_coords_names="z")
    data.x.attrs = {"units": "µm"}
    data.y.attrs = {"units": "µm"}
    data.bz.attrs = {"long_name": "vertical magnetic field", "units": "nT"}
    
    
    noise_std_dev = 50  # nT
    data.bz.values += mg.gaussian_noise(noise_std_dev, data.bz.shape, seed=42)
    data.bz.values += (np.random.randint(-2000, 2000)) # nT

    # Exportar cada arquivo para a pasta
    filename = os.path.join(output_folder, f"data_simulation_{i}.nc")
    data.to_netcdf(
        filename,
        encoding={
            "bz": {"zlib": True, "complevel": 9},  # Compactação máxima
            "x": {"zlib": True, "complevel": 9},
            "y": {"zlib": True, "complevel": 9},
        },
    )
    


In [4]:
# Compactar a pasta em um arquivo ZIP
zip_filename = "simulations.zip"
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_folder))

In [5]:
shutil.rmtree(output_folder)

In [6]:
print(mg.vector_to_angles(np.sum(true_vector, axis=0)))

(31.12941152350789, 30.51331346108358, 3.2652968198910724e-10)
